# DuckDuckGo搜索

本笔记本提供了[快速入门DuckDuckGo搜索](/docs/integrations/tools/)的快速概览。如需查看所有DuckDuckGo搜索功能和配置的详细文档，请前往[API参考](https://api.js.langchain.com/classes/langchain_community_tools_duckduckgo_search.DuckDuckGoSearch.html)。

DuckDuckGo搜索提供了一款以隐私为中心的搜索API，专为LLM代理设计。它能够与广泛的数据源无缝集成，在提供相关搜索结果的同时优先保护用户隐私。

## 概述

### 集成详情

| 类别 | 包 | [PY支持](https://python.langchain.com/docs/integrations/tools/ddg/) | 最新包 |
| :--- | :--- | :---: | :---: |
| [DuckDuckGo搜索](https://api.js.langchain.com/classes/langchain_community_tools_duckduckgo_search.DuckDuckGoSearch.html) | [`@langchain/community`](https://www.npmjs.com/package/@langchain/community) | ✅ |  ![NPM - 版本](https://img.shields.io/npm/v/@langchain/community?style=flat-square&label=%20&) |

## 安装设置

该集成位于`@langchain/community`包中，并依赖于`duck-duck-scrape`包：

```{=mdx}
import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/community @langchain/core duck-duck-scrape
</Npm2Yarn>
```

### 凭证

为了获得最佳的可观察性，设置[LangSmith](https://smith.langchain.com/)也是有帮助的（但不是必须的）：

```typescript
process.env.LANGSMITH_TRACING="true"
process.env.LANGSMITH_API_KEY="your-api-key"
```

## 实例化

你可以这样实例化一个 `DuckDuckGoSearch` 工具的实例：

In [1]:
import { DuckDuckGoSearch } from "@langchain/community/tools/duckduckgo_search"

const tool = new DuckDuckGoSearch({ maxResults: 1 })

## 调用

### [使用参数直接调用](/docs/concepts/tools)

In [2]:
await tool.invoke("what is the current weather in sf?")

[{"title":"San Francisco, CA Current Weather | AccuWeather","link":"https://www.accuweather.com/en/us/san-francisco/94103/current-weather/347629","snippet":"<b>Current</b> <b>weather</b> <b>in</b> San Francisco, CA. Check <b>current</b> conditions in San Francisco, CA with radar, hourly, and more."}]


### [通过工具调用调用](/docs/concepts/tools)

我们还可以使用模型生成的 `ToolCall` 来调用工具，在这种情况下将返回一个 `ToolMessage`：

In [3]:
// This is usually generated by a model, but we'll create a tool call directly for demo purposes.
const modelGeneratedToolCall = {
  args: {
    input: "what is the current weather in sf?"
  },
  id: "tool_call_id",
  name: tool.name,
  type: "tool_call",
}
await tool.invoke(modelGeneratedToolCall)

ToolMessage {
  "content": "[{\"title\":\"San Francisco, CA Weather Conditions | Weather Underground\",\"link\":\"https://www.wunderground.com/weather/us/ca/san-francisco\",\"snippet\":\"San Francisco <b>Weather</b> Forecasts. <b>Weather</b> Underground provides local & long-range <b>weather</b> forecasts, weatherreports, maps & tropical <b>weather</b> conditions for the San Francisco area.\"}]",
  "name": "duckduckgo-search",
  "additional_kwargs": {},
  "response_metadata": {},
  "tool_call_id": "tool_call_id"
}


## 链式调用

我们可以通过首先将工具绑定到一个[工具调用模型](/docs/how_to/tool_calling/)，然后调用它，以实现链式调用：

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```


In [1]:
// @lc-docs-hide-cell

import { ChatOpenAI } from "@langchain/openai"

const llm = new ChatOpenAI({
  model: "gpt-4o-mini",
})

In [10]:
import { HumanMessage } from "@langchain/core/messages";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { RunnableLambda } from "@langchain/core/runnables";

const prompt = ChatPromptTemplate.fromMessages(
  [
    ["system", "You are a helpful assistant."],
    ["placeholder", "{messages}"],
  ]
)

const llmWithTools = llm.bindTools([tool]);

const chain = prompt.pipe(llmWithTools);

const toolChain = RunnableLambda.from(
  async (userInput: string, config) => {
    const humanMessage = new HumanMessage(userInput,);
    const aiMsg = await chain.invoke({
      messages: [new HumanMessage(userInput)],
    }, config);
    const toolMsgs = await tool.batch(aiMsg.tool_calls, config);
    return chain.invoke({
      messages: [humanMessage, aiMsg, ...toolMsgs],
    }, config);
  }
);

const toolChainResult = await toolChain.invoke("how many people have climbed mount everest?");

In [11]:
const { tool_calls, content } = toolChainResult;

console.log("AIMessage", JSON.stringify({
  tool_calls,
  content,
}, null, 2));

AIMessage {
  "tool_calls": [],
  "content": "As of December 2023, a total of 6,664 different people have reached the summit of Mount Everest."
}


## 代理

关于如何在代理中使用LangChain工具的指南，请参阅 [LangGraph.js](https://langchain-ai.github.io/langgraphjs/) 文档。

## API 参考文档

如需详细了解所有 DuckDuckGoSearch 功能和配置，请访问 [API 参考文档](https://api.js.langchain.com/classes/langchain_community_tools_duckduckgo_search.DuckDuckGoSearch.html)